# JsonOutputParser

- JsonOutputParser는 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 도구
- 이 도구는 Large Language Model (LLM)이 데이터를 조회하고 결과를 도출할 때, 지정된 스키마에 맞게 JSON 형식으로 데이터를 반환할 수 있도록 설계

- LLM이 데이터를 정확하고 효율적으로 처리하여 사용자가 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량(예: 인텔리전스)이 충분히 커야한다.
- 예를 들어, llama-70B 모델은 llama-8B 모델보다 더 큰 용량을 가지고 있어 보다 복잡한 데이터를 처리하는 데 유리

## 참고

- `JSON (JavaScript Object Notation)` 은 데이터를 저장하고 구조적으로 전달하기 위해 사용되는 경량의 데이터 교환 포맷
- 웹 개발에서 매우 중요한 역할을 하며, 서버와 클라이언트 간의 통신을 위해 널리 사용
- JSON은 읽기 쉽고, 기계가 파싱하고 생성하기 쉬운 텍스트를 기반

- JSON의 기본 구조
  - JSON 데이터는 이름(키)과 값의 쌍
  - 여기서 "이름"은 문자열이고, "값"은 다양한 데이터 유형일 수 있다.
  - JSON은 두 가지 기본 구조
    - 객체: 중괄호 {}로 둘러싸인 키-값 쌍의 집합. 각 키는 콜론 :을 사용하여 해당하는 값과 연결되며, 여러 키-값 쌍은 쉼표 ,로 구분
    - 배열: 대괄호 []로 둘러싸인 값의 순서 있는 목록. 배열 내의 값은 쉼표 ,로 구분

```json
{
  "name": "John Doe",
  "age": 30,
  "is_student": false,
  "skills": ["Java", "Python", "JavaScript"],
  "address": {
    "street": "123 Main St",
    "city": "Anytown"
  }
}
```

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_teddynote import logging
from dotenv import load_dotenv

load_dotenv()
logging.langsmith("CH03-OutputParser")
model = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

- `JsonOutputParser`를 사용하여 파서를 설정하고, 프롬프트 템플릿에 지시사항을 주입

In [5]:
question = "지구 온난화의 심각성 대해 알려주세요."

parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [8]:
# 프롬프트 템플릿을 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
answer = chain.invoke({"question": question})

'지구 온난화는 지구 평균 기온이 상승하여 극심한 기후 변화, 해수면 상승, 생태계 파괴 등 심각한 환경 문제를 초래합니다.'

In [12]:
# 타입을 확인
type(answer)

dict

In [13]:
answer['description']

'지구 온난화는 지구 평균 기온이 상승하여 극심한 기후 변화, 해수면 상승, 생태계 파괴 등 심각한 환경 문제를 초래합니다.'